# Пример использования для доразметки установленного локально (а потому без абонентской платы) ИИ.

In [ ]:
Тайминги получены на ПК с ОЗУ 32ГБ + ГПУ RTX3060.

In [1]:
#https://habr.com/ru/articles/775870/
#https://habr.com/ru/articles/769124/

In [2]:
#!pip install protobuf

In [3]:
#!pip install --upgrade langchain

In [4]:
#!pip install sentencepiece

In [5]:
#!pip install transformers accelerate peft

In [6]:
#Могут возникать ошибки из-за несовместимости версий
#pip install peft==0.10.0

In [7]:
%%time
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

MODEL_NAME = "IlyaGusev/saiga_mistral_7b_lora"

# Загружаем модель
config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
	config.base_model_name_or_path,
	torch_dtype=torch.float16,
	device_map="auto"
)
model = PeftModel.from_pretrained(
	model,
	MODEL_NAME,
	torch_dtype=torch.float16
)
model.eval()

# Определяем токенайзер
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

# Функция для обработки запросов
def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    #print(data)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    #print(output_ids)
    output_ids = output_ids[len(data["input_ids"][0]):]
    #print(output_ids)
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

/home/dm/anaconda3/envs/p310implicit/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
/home/dm/anaconda3/envs/p310implicit/lib/python3.10/site-packages/accelerate/utils/modeling.py:1462: UserWarning: Current model requires 520101632 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Some parameters are on the meta device because they were offloaded to the cpu.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


CPU times: user 11.3 s, sys: 18.8 s, total: 30.1 s
Wall time: 43.3 s


# Пример галлюцинации

In [8]:
%%time
# Формируем запрос
#PROMT_TEMPLATE = '<s>system\nТы — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.</s><s>user\n{inp}</s><s>bot\n'
PROMT_TEMPLATE = '<s>user\n{inp}</s><s>bot\n'
inp = 'Где находятся панамские острова?'
prompt = PROMT_TEMPLATE.format(inp=inp)

# Отправляем запрос в llm
output = generate(model, tokenizer, prompt, generation_config)

print(output)

Панамские острова расположены в Тихоокеанском регионе, примерно в 100 километрах к юго-востоку от Панамы. Они являются частью архипелага Сан-Блас, который включает в себя более чем 400 островов и атоллов.
CPU times: user 1min 4s, sys: 1.76 s, total: 1min 6s
Wall time: 1min 6s


# Заполнение пустой категории

In [9]:
%%time
# Формируем запрос
#PROMT_TEMPLATE = '<s>system\nТы — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.</s><s>user\n{inp}</s><s>bot\n'
PROMT_TEMPLATE = '<s>user\n{inp}</s><s>bot\n'
inp = 'Составь список товаров категории молочные продукты.'
prompt = PROMT_TEMPLATE.format(inp=inp)

# Отправляем запрос в llm
output = generate(model, tokenizer, prompt, generation_config)

print(output)

1. Молоко
2. Сметана
3. Кефир
4. Йогурт
5. Мороженое
6. Масло
7. Сиреневый сыр
8. Капучино
9. Латте
10. Млечный коктейль
CPU times: user 53.4 s, sys: 1.38 s, total: 54.8 s
Wall time: 54.8 s


# Проверка соответствия товара к категории

In [10]:
%%time
# Формируем запрос
PROMT_TEMPLATE = '<s>user\n{inp}</s><s>bot\n'
inp = 'Относится ли кефир к молочным продуктам? Варианты ответа - да или нет.'
prompt = PROMT_TEMPLATE.format(inp=inp)

# Отправляем запрос в llm
output = generate(model, tokenizer, prompt, generation_config)

print(output)

да
CPU times: user 1.52 s, sys: 57.2 ms, total: 1.58 s
Wall time: 1.6 s


# IlyaGusev/saiga_yandexgpt_8b

In [1]:
#!pip install accelerate
#!pip install -i https://pypi.org/simple/ bitsandbytes

In [1]:
%%time
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

MODEL_NAME = "IlyaGusev/saiga_yandexgpt_8b"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_8bit=True,
    torch_dtype=torch.bfloat16,
    #device_map="auto"
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

inputs = ["Почему трава зеленая?", "Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч"]
for query in inputs:
    prompt = tokenizer.apply_chat_template([{
        "role": "user",
        "content": query
    }], tokenize=False, add_generation_prompt=True)
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    data.pop("token_type_ids", None)
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    print(query)
    print(output)
    print()
    print("==============================")
    print()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 128009,
  "max_length": 1024000,
  "max_new_tokens": 2048,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.6,
  "top_p": 0.95
}

Почему трава зеленая?
Трава зеленая благодаря наличию в ее клетках особого пигмента, называемого хлорофиллом. Этот пигмент играет ключевую роль в процессе фотосинтеза – превращении солнечного света в энергию для роста и развития растений. Хлорофилл поглощает свет преимущественно в синей и красной частях спектра (400–500 нм и 620–730 нм соответственно), но отражает зеленый свет (около 500–570 нм). Именно поэтому мы видим траву зеленой: наше зрение воспринимает отраженный зеленый свет как зеленый цвет.


Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч
В одном уютном городке на окраине России жила девочка по имени Таня. Её дом стоял среди старых лип и яблонь, а за окном всегда играли солнечные лучи, освещая комнату тёплым светом. В 

In [5]:
%%time
inputs = ["Относится ли 'ряженка' к 'молочным продуктам', да или нет?", 
          "Относится ли 'снежок' к 'молочным продуктам'? Варианты ответа - да или нет.",
          "Составь список товаров категории молочные продукты."]
for query in inputs:
    prompt = tokenizer.apply_chat_template([{
        "role": "user",
        "content": query
    }], tokenize=False, add_generation_prompt=True)
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    data.pop("token_type_ids", None)
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    print(query)
    print(output)
    print()
    print("==============================")
    print()

Относится ли 'ряженка' к 'молочным продуктам', да или нет?
Да, ряженка относится к молочным продуктам. Это кисломолочный продукт, получаемый путём сквашивания молока с использованием специальных заквасок, включающих болгарскую палочку и термофильный стрептококк.


Относится ли 'снежок' к 'молочным продуктам'? Варианты ответа - да или нет.
Нет, "снежок" не относится к молочным продуктам. Это разговорное название для снежного шара (сферического устройства с механизмом вращения, внутри которого находится снег), а также может обозначать различные виды сладостей на основе снега или мороженого. Однако в контексте молочных продуктов он не используется.


Составь список товаров категории молочные продукты.
1. Молоко (цельное, обезжиренное, ультрапастеризованное)
2. Сливки (жирные и нежирные)
3. Сметана (различной жирности)
4. Йогурт (натуральный, фруктовый, ягодный, овощной, бифидо-йогурты)
5. Кефир (традиционный, биокефир, кефир с добавками)
6. Творог (классический, зерненый, обезжиренный, тв